In [1]:
from autocnet_server.graph.graph import NetworkCandidateGraph
from autocnet_server.db.model import Network
from autocnet_server.config import AutoCNet_Config as config

from sqlalchemy.orm import aliased, create_session, scoped_session, sessionmaker
from sqlalchemy import create_engine

In [2]:
sc = config()
db_uri = 'postgresql://{}:{}@{}:{}/{}'.format(sc.database_username,
                                              sc.database_password,
                                              sc.database_host,
                                              sc.database_port,
                                              sc.database_name)
engine = create_engine(db_uri)
connection = engine.connect()
session = sessionmaker(bind=engine)()

## Go from a network to an ISIS3 Control Network

In [3]:
import numpy as np
import pandas as pd
from plio.io.io_controlnetwork import to_isis, write_filelist

# 'point_id', 'x', 'y', 'image_index']

In [4]:
ncg = NetworkCandidateGraph.from_database()

In [5]:
serials = ncg.serials()
for k, v in serials.items():
    serials[k] = v.replace('Mars_Reconnaissance_Orbiter', 'MRO')

In [23]:
for i, n in ncg.nodes(data='data'):
    if 'D17_033925_1884_XN_08N199W.cal.cub' in n['image_name']:
        print(i)
        break

1092


In [14]:
data = []
for i, r in enumerate(session.query(Network)):
    serial_number = serials[r.image_id]
    data.append([r.point_id, 2, serial_number, 2, r.x, r.y, r.image_id])
df = pd.DataFrame(data, columns=['point_id','point_type', 'serialnumber', 'measure_type', 'x', 'y','image_index'])

In [27]:
imgs = np.unique(df.image_index.values)

In [51]:
f = np.unique(df.head(3200).image_index.values)

In [28]:
flist = [n['image_path'] for i, n in ncg.nodes(data='data') if i in imgs]

In [46]:
[(i, n['image_path']) for i, n in ncg.nodes(data='data') if i in f]

[(688, '/scratch/jlaura/ctx/cal/D14_032501_1900_XN_10N202W.cal.cub'),
 (814, '/scratch/jlaura/ctx/cal/P17_007553_1835_XN_03N203W.cal.cub'),
 (721, '/scratch/jlaura/ctx/cal/F04_037182_1886_XN_08N200W.cal.cub'),
 (943, '/scratch/jlaura/ctx/cal/P19_008410_1830_XI_03N205W.cal.cub'),
 (1048, '/scratch/jlaura/ctx/cal/P17_007566_1872_XN_07N198W.cal.cub'),
 (789, '/scratch/jlaura/ctx/cal/P02_001870_1869_XN_06N206W.cal.cub'),
 (717, '/scratch/jlaura/ctx/cal/G20_025908_1897_XN_09N206W.cal.cub'),
 (1116, '/scratch/jlaura/ctx/cal/G14_023811_1870_XN_07N197W.cal.cub'),
 (684, '/scratch/jlaura/ctx/cal/D02_028031_1907_XN_10N203W.cal.cub'),
 (689, '/scratch/jlaura/ctx/cal/G20_025934_1911_XI_11N197W.cal.cub'),
 (1094, '/scratch/jlaura/ctx/cal/P18_008146_1899_XN_09N196W.cal.cub'),
 (914, '/scratch/jlaura/ctx/cal/J04_046387_1839_XN_03N210W.cal.cub'),
 (1139, '/scratch/jlaura/ctx/cal/G18_025367_1858_XI_05N197W.cal.cub'),
 (800, '/scratch/jlaura/ctx/cal/F02_036602_1909_XN_10N204W.cal.cub'),
 (891, '/scratch

In [15]:
for i, g in df.groupby('point_id'):
    for j, h in g.groupby('serialnumber'):
        if len(h) > 1:
            print(h)

In [16]:
for i, g in df.groupby('point_id'):
    if len(g) < 2:
        print(g)
        break

In [29]:
write_filelist(flist,'ctx.lis')

In [43]:
ndf = df[~df.index.isin(df.query('(x < 0) or  (y < 0)').index)]

In [45]:
for i, g in ndf.groupby('point_id'):
    if len(g) < 2:
        print(g)
        break

In [77]:
to_isis('/home/jlaura/autocnet_server/notebooks/ctx.net', ndf, serials, mode='wb', targetname='Mars')
write_filelist(flist,'ctx.lis')

30823


In [50]:
sizes = {}
for i, n in ncg.nodes(data='data'):
    sizes[i] = n.geodata.raster_size

In [76]:
ndf[ndf.x > 100000]

,point_id,point_type,serialnumber,measure_type,x,y,image_index


In [72]:
def get_ext(r, sizes):
    ext = sizes[r.image_index]
    if r.x > ext[0] or r.y > ext[1]:
        return False
    return True
ext_mask = ndf.apply(get_ext, args=(sizes,), axis=1)
ext_mask

0         True
1         True
2         True
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15        True
16        True
17        True
18        True
19        True
20        True
21        True
22        True
23        True
24        True
25        True
26        True
27        True
28        True
29        True
          ... 
148251    True
148252    True
148253    True
148254    True
148255    True
148256    True
148257    True
148258    True
148259    True
148260    True
148261    True
148262    True
148263    True
148264    True
148265    True
148266    True
148267    True
148268    True
148269    True
148270    True
148271    True
148272    True
148273    True
148275    True
148276    True
148277    True
148278    True
148279    True
148280    True
148281    True
Length: 146110, dtype: bool

In [74]:
ndf = ndf[ext_mask]

In [78]:
for i, g in ndf.groupby('point_id'):
    if len(g) < 2:
        print(g)
        break

In [80]:
len(ndf)

143970